*2021/01/22 Fri*

<hr>

# 13-1. 객체의 유일한 소유권 - unique_ptr

C++ 11에서 자원을 관리하는 방법에 대해서 다룬다.
컴퓨터에서 자원이라 하면, 할당한 메모리도 자원이고 open한 파일 역시 하나의 자원이라 할 수 있다.

## 자원 관리의 중요성

C++ 이후에 나온 많은 언어(Java 등등)들은 대부분 가비지 컬렉터라 불리는 자원 청소기가 기본적으로 내장되어 있다. 그러나 C++은 그렇지 않다. 프로그램 종료 시까지 영원히 남아 있게 된다.

아래와 같은 상황을 어떻게 해결해야 할까?

In [ ]:
#include <iostream>

class A {
    int *data;
    
public:
    A() {
        data = new int[100];
        std::cout << "자원을 획득함!" << std::endl;
    }
    ~A() {
        std::cout << "자원을 해제함!" << std::endl;
        delete[] data;
    }
};

void thrower() {
    // 예외를 발생시킴!
    throw 1;
}

void do_something() {
    A *pa = new A();
    thrower();
    
    // 발생된 예외로 인해 delete pa가 호출되지 않는다!
    delete pa;
}

int main() {
    try {
        do_something();
    } catch (int i) {
        std::cout << "예외 발생!" << std::endl;
        // 예외 처리
    }
}

## Resource Acqusition Is Initialization - RAII

비야네 스트로스트룹은 C++에서 자원을 관리하는 방법으로 다음과 같은 디자인 패턴을 제안하였다. RAII; 자원의 획득은 초기화다.

자원 관리를 스택에 할당한 객체를 통해 수행하는 것.

이전 강좌에서, 예외가 발생해서 함수를 빠져나가더라도, 그 함수의 스택에 정의되어 있는 모든 객체들은 빠집없이 소멸자가 호출된다고 했다.(이를 stack unwinding이라 했다.) 예외가 발생하지 않는다면, 함수가 종료될 때 당연히 소멸자들이 호출되는 것.

**그러면 이 소멸자들 안에 다 사용한 자원을 해제하는 루틴을 넣으면 어떨까?**

예로 들어, 위 포인터 `pa`의 경우 객체가 아니므로 소멸자가 호출되지 않는다. 그러면 그 대신에 `pa`를 일반적인 포인터가 아닌, 포인터 '객체'로 만들어서 자신이 소멸될 때 자신이 가리키고 있는 데이터도 같이 delete하게 하면 되는 것.

즉, 자원(이 경우 메모리) 관리를 스택의 객체(포인터 객체)를 통해 수행하게 되는 것이다.

이렇게 똑똑하게 작동하는 포인터 객체를 <u>스마트 포인터</u>라 부른다. C++ 11 이전에는 `auto_ptr`란 게 잠시 등장했지만, 많은 문제가 있어 사용을 금지.

이를 보완한 두 가지 형태의 새로운 스마트 포인터가 `unique_ptr`과 `shared_ptr`.

## 객체의 유일한 소유권 - unique_ptr

C++에서 메모리를 잘못된 방식으로 관리하면, 크게 두 가지 종류의 문제점이 발생할 수 있다.

1. 메모리를 사용한 후 해제하지 않은 경우(memory leak) : 서버처럼 장시간 작동하는 프로그램의 경우 나중에 서버가 죽을 수 있다. 그런데, 다행히 RAII 패턴을 사용하면 해결할 수 있고, RAII를 통해서 사용이 끝난 메모리는 항상 해제시켜 버리면 메모리 누수 문제를 사전에 막을 수 있다.
2. 해제된 메모리를 다시 참조하는 경우

In [ ]:
Data *data = new Data();
Data *data2 = data;

// data의 입장 : 사용 다 했으니 소멸시켜야지.
delete data;

// ...

// data2의 입장 : 나도 사용 다 했으니 소멸시켜야지.
delete data2;

이 경우, 메모리 오류가 나면서 프로그램이 죽는다. <u>double free 버그</u>.

이 문제가 발생한 이유는, 만들어진 객체의 소유권이 명확하지 않아서이다. 만약, 어떤 포인터에 객체의 유일한 소유권을 부여해서, 이 포인터 말고는 객체를 소멸시킬 수 없다.라고 한다면, 이런 일은 발생하지 않을 것.

이렇게 특정 객체에 유일한 소유권을 부여하는 포인터 객체를 `unique_ptr`이라 한다.

In [ ]:
#include <iostream>
#include <memory>

class A {
    int *data;
    
public:
    A() {
        std::cout << "자원을 획득함!" << std::endl;
        data = new int[100];
    }
    
    void some() { std::cout << "일반 포인터와 동일하게 사용 가능!" << std::endl; }
    
    ~A() {
        std::cout << "자원을 해제함!" << std::endl;
        delete[] data;
    }
};

void do_something() {
    std::unique_ptr<A> pa(new A());  // A *pa = new A();와 동일한 문장이라 보면 된다.
    pa->some();
}

int main() { do_something(); }

\- 실행 결과
```
자원을 획득함!
일반 포인터와 동일하게 사용 가능!
자원을 해제함!
```

`unique_ptr`은 `->` 연산자를 오버로드해서 마치 포인터를 다루는 것과 같이 사용할 수 있게 하였다.

또한 `unique_ptr` 덕분에 `RAII` 패턴을 사용할 수 있다. `pa`는 스택에 정의된 객체이므로 `do_something()` 함수가 종료될 때 자동으로 소멸자가 호출된다. 그리고 이 `unique_ptr`은 소멸자 안에서 자신이 가리키고 있는 자원을 해제해주므로 자원이 잘 해제될 수 있었다.

Q. 만약 `unique_ptr`을 복사하려고 한다면?

In [ ]:
void do_something() {
    std::unique_ptr<A> pa(new A());
    
    // pb도 객체를 가리키게 할 수 있을까?
    std::unique_ptr<A> pb = pa;
}

\- 실행 결과
```
'std::unique_ptr<A,std::default_delete<_Ty>>::unique_ptr(const std::unique_ptr<_Ty,std::default_delete<_Ty>> &)': attempting to reference a deleted function
```

컴파일 오류! 삭제된 함수를 사용하려고 했다는 것.

## 삭제된 함수

사용을 원치 않는 함수를 삭제시키는 방법은 C++ 11에 추가된 기능이다.

In [ ]:
#include <iostream>

class A {
public:
    A(int a) {}
    A(const A& a) = delete;  // 삭제된 함수
};

int main() {
    A a(3);  // 가능
    A b(a);  // 불가능 (복사 생성자는 삭제됨) -> 컴파일 오류
}

복사 생성자를 명시적으로 삭제했으므로 컴파일 오류.

`= delete;`를 사용하게 되면, 프로그래머가 명시적으로 '이 함수는 쓰지 마!'라고 표현할 수 있다. 혹시나 사용하더라도 컴파일 오류 발생.

`unique_ptr`도 마찬가지로 `unique_ptr`의 복사 생성자가 명시적으로 삭제되었음.(디폴트 복사 생성자 없음.) 어떠한 객체를 유일하게 소유해야 하므로. 안 그러면 `double free` 버그가 발생하게 된다.

## unique_ptr 소유권 이전하기

복사는 안 되지만 소유권은 이전할 수 있다.

In [ ]:
#include <iostream>
#include <memory>

class A {
    int *data;
    
public:
    A() {
        std::cout << "자원을 획득함!" << std::endl;
        data = new int[100];
    }
    
    void some() { std::cout << "일반 포인터와 동일하게 사용가능!" << std::endl; }
    
    ~A() {
        std::cout << "자원을 해제함!" << std::endl;
        delete[] data;
    }
};

void do_something() {
    std::unique_ptr<A> pa(new A());
    std::cout << "pa : ";
    pa->some();
    
    // pb에 소유권을 이전.
    std::unique_ptr<A> pb = std::move(pa);
    std::cout << "pb : ";
    pb->some();
}

int main() { do_something(); }

```
자원을 획득함!
pa : 일반 포인터와 동일하게 사용 가능!
pb : 일반 포인터와 동일하게 사용 가능!
자원을 해제함!
```

`unique_ptr`은 복사 생성자는 정의되어 있지 않지만, 이동 생성자는 가능하다.

```
std::unique_ptr<A> pb = std::move(pa);

pa.get();  // 0 (nullptr)이 나온다!
```

소유권이 이전된 `unique_ptr`을 dangling pointer라 하며, 이를 재참조할 시에는 런타임 오류가 발생한다.

## unique_ptr를 함수 인자로 전달하기

In [ ]:
#include <iostream>
#include <memory>

class A {
    int *data;
    
public:
    A() {
        std::cout << "자원을 획득함!" << std::endl;
        data = new int[100];
    }
    
    void some() { std::cout << "일반 포인터와 동일하게 사용가능!" << std::endl; }
    
    void do_sth(int a) {
        std::cout << "무언가를 한다!" << std::endl;
        data[0] = a;
    }
    
    ~A() {
        std::cout << "자원을 해제함!" << std::endl;
        delete[] data;
    }
};

// 올바르지 않은 전달 방식
void do_something(std::unique_ptr<A>& ptr) { ptr->do_sth(3); }

int main() {
    std::unique_ptr<A> pa(new A());
    do_something(pa);
}

이렇게 하면 문맥상 맞지 않다.

`ptr`은 레퍼런스이므로 `do_something` 함수가 종료되면서 `pa`가 가리키고 있는 객체를 파괴하진 않지만, `pa`가 유일하게 소유하고 있던 객체는 이제 적어도 `pa`가 `do_something` 함수 내부에서는 `ptr`을 통해서 소유할 수 있게 된다. 즉, `unique_ptr`은 소유권을 의미한다는 원칙에 위배되는 것.

따라서, `unique_ptr`의 레퍼런스를 사용하는 것은 소유권이라는 중요한 의미를 망각한 채 단순히 포인터의 단순한 `Wrapper`로 사용하는 것에 불과하다. 그러면 `unique_ptr`을 전달하는 방법은? 그냥 단순하게 원래의 포인터 주소값을 전달해 주면 된다!

In [ ]:
#include <iostream>
#include <memory>

class A {
    int *data;

public:
    A() {
        std::cout << "자원을 획득함!" << std::endl;
        data = new int[100];
    }
    
    void some() { std::cout << "일반 포인터와 동일하게 사용가능!" << std::endl; }
    
    void do_sth(int a) {
        std::cout << "무언가를 한다!" << std::endl;
        data[0] = a;
    }
    
    ~A() {
        std::cout << "자원을 해제함!" << std::endl;
        delete[] data;
    }
};

void do_something(A *ptr) { ptr->do_sth(3); }

int main() {
    std::unique_ptr<A> pa(new A());
    do_something(pa.get());  // 포인터 전달!
}

\- 정리!
* `unique_ptr`은 어떤 객체의 유일한 소유권을 나타내는 포인터. `unique_ptr`이 소멸될 때, 가리키던 객체 역시 소멸된다.
* 만약에 다른 함수에서 `unique_ptr`가 소유한 객체에 일시적으로 접근하고 싶다면, `get`을 통해 해당 객체의 포인터를 전달하면 된다.
* 만약 소유권을 이동하고자 한다면, `unique_ptr`을 `move`하면 된다.

## unique_ptr을 쉽게 생성하기

C++ 14부터 `unique_ptr`을 간단히 만들 수 있는 `std::make_unique` 함수를 제공한다.

In [ ]:
#include <iostream>
#include <memory>

class Foo {
    int a, b;
    
public:
    Foo(int a, int b) : a(a), b(b) { std::cout << "생성자 호출!" << std::endl; }
    void print() { std::cout << "a : " << ", b : " << b << std::endl; }
    ~Foo() { std::cout << "소멸자 호출!" << std::endl; }
};

int main() {
    auto ptr = std::make_unique<Foo>(3, 5);
    // make_unique 함수는 아예 템플릿 인자로 전달된 클래스의 생성자에, 인자들에 직접 완벽한 전달을 수행한다.
    // std::unique_ptr<Foo> ptr(new Foo(3, 5));
    ptr->print();
}

## unique_ptr를 원소로 가지는 컨테이너

`unique_ptr`을 원소로 가지는 `STL` 컨테이너. 다른 타입과 큰 차이는 없지만, 복사 생성자가 없다는 특성 때문에 처음 사용하는 경우 애를 먹는 경우가 종종 있다.

아래처럼 하면 무시무시한 컴파일 오류를 맛보게 된다.

In [ ]:
#include <iostream>
#include <memory>
#include <vector>

class A {
    int *data;
    
public:
    A(int i) {
        std::cout << "자원을 획득함!" << std::endl;
        data = new int[100];
        data[0] = i;
    }
    
    void some() { std::cout << "일반 포인터와 동일하게 사용 가능!" << std::endl; }
    
    ~A() {
        std::cout << "자원을 해제함!" << std::endl;
        delete[] data;
    }
};

int main() {
    std::vector<std::unique_ptr<A>> vec;
    std::unique_ptr<A> pa(new A(1));
    
    vec.push_back(pa);  // 컴파일 오류
}

왜냐하면 삭제된 `unique_ptr`의 복사 생성자에 접근하였기 때문. 기본적으로 `vector`의 `push_back` 함수는 전달된 인자를 복사해서 집어넣기 때문에 문제가 발생하는 것이다.

이를 방지하기 위해서는 아래와 같이 명시적으로 `pa`를 `vector` 안으로 이동시켜 주어야 한다. 즉, `push_back`의 우측 값 레퍼런스를 받는 버전이 오버로딩될 수 있도록.

In [ ]:
int main() {
    std::vector<std::unique_ptr<A>> vec;
    std::unique_ptr<A> pa(new A(1));
    
    vec.push_back(std::move(pa));  // 잘 실행됨
}

하지만 재미있게도 `emplace_back()`을 이용하면 `vector` 안에 `unique_ptr`을 직접 생성하면서 집어넣을 수도 있다. 즉, 불필요한 이동 과정을 생략할 수 있다.

In [ ]:
#include <iostream>
#include <memory>
#include <vector>

class A {
    int *data;
    
public:
    A(int i) {
        std::cout << "자원을 획득함!" << std::endl;
        data = new int[100];
        data[0] = i;
    }
    
    void some() { std::cout << "값 : " << data[0] << std::endl; }
    
    ~A() {
        std::cout << "자원을 해제함!" << std::endl;
        delete[] data;
    }
};

int main() {
    std::vector<std::unique_ptr<A>> vec;
    
    // vec.push_back(std::unique_ptr<A>(new A(1))); 과 동일
    vec.emplace_back(new A(1));
    
    vec.back()->some();
}

```
자원을 획득함!
값 : 1
자원을 해제함!
```

`emplace_back` 함수는 전달된 인자를 완벽한 전달(perfect forwarding)을 통해 직접 `unique_ptr<A>`의 생성자에 전달해서, `vector` 맨 뒤에 `unique_ptr<A>` 객체를 생성해버린다. 따라서 위처럼 불필요한 이동 연산이 필요없게 된다. (왜냐하면 `vector` 맨 뒤에 생성하기 때문에!)

참고로 `emplace_back`을 사용 시에 어떠한 생성자가 호출되는지 주의해야 한다.

In [ ]:
std::vector<int> v;
v.emplace_back(100000);  // 100000이란 int 값을 v에 추가.

std::vector<std::vector<int>> v;
v.emplace_back(100000);  // 원소가 100000개 들어있는 벡터를 v에 추가